In [1]:
from config_paths import *
import sys
sys.path.append(str(SRC_DIR))

from fuzzy_sql.fuzzy_sql import *
import json
import os
from pathlib import Path

DATASET_NAME='sdgd'
import glob

In [2]:
# set directories
metadata_dir = os.path.join(DATA_DIR, DATASET_NAME,'processed/metadata')
real_dir = os.path.join(DATA_DIR,DATASET_NAME, 'processed/real')
syn_dir = os.path.join(DATA_DIR,DATASET_NAME, 'processed/synthetic')
db_path = os.path.join(DB_DIR, f'{DATASET_NAME}.db')

In [3]:
# poplulate filenames from folder into a list
real_fnames=glob.glob(real_dir+'/*.csv')
real_fnames=[Path(real_fname).stem for real_fname in real_fnames]
real_fnames.sort()


# generate filenames
meta_jsons=[]
real_csvs=[]
for real_fname in real_fnames:
    meta_jsons.append(real_fname+'.json')
    real_csvs.append(real_fname+'.csv')
    


In [4]:
# connect to db
conn = sqlite3.connect(db_path)

In [5]:
# import real data into database
for real_csv,meta_json in zip(real_csvs,meta_jsons):
    #Import real
    data,_=prep_data_for_db(os.path.join(real_dir,real_csv)) # Note: This function  can be used to quickly generate metadata template, but this is not shown here.
    with open(os.path.join(metadata_dir, meta_json), 'r') as f:
        metadata=json.load(f)
    print(f"Processing {real_csv} and {meta_json} ")
    candidate_db_idx=get_vars_to_index(metadata,data)
    table_name=Path(real_csv).stem
    make_table(table_name, data, conn,indx_vars=candidate_db_idx)

Processing C1.csv and C1.json 
Table C1 already exists in the database
Processing C10.csv and C10.json 
Table C10 already exists in the database
Processing C11.csv and C11.json 
Table C11 already exists in the database
Processing C12.csv and C12.json 
Table C12 already exists in the database
Processing C13.csv and C13.json 
Table C13 already exists in the database
Processing C14.csv and C14.json 
Table C14 already exists in the database
Processing C15.csv and C15.json 
Table C15 already exists in the database
Processing C16.csv and C16.json 
Table C16 already exists in the database
Processing C17.csv and C17.json 
Table C17 already exists in the database
Processing C18.csv and C18.json 
Table C18 already exists in the database
Processing C19.csv and C19.json 
Table C19 already exists in the database
Processing C2.csv and C2.json 
Table C2 already exists in the database
Processing C20.csv and C20.json 
Table C20 already exists in the database
Processing C21.csv and C21.json 
Table C21 a

In [6]:
def find_syn_fnames(syn_data_dir: Path, real_names: list) -> dict:
    syn_dict={}
    for real_name_i in real_names:
        catch_file=[]
        for syn_name_i in os.listdir(syn_data_dir):
            if syn_name_i.startswith(real_name_i+'_'):
                catch_file.append(Path(syn_name_i).stem)
        syn_dict[real_name_i]=catch_file
    print('Extracted the names of all available synthetic datasets corresponding to {} real datasets'.format(str(len(syn_dict))))
    return syn_dict



In [7]:
# import syn data into database
syn_dict=find_syn_fnames(syn_dir,real_fnames)

#drop datasets with no synthetic data available 
tabular_test_set={key:val for key,val in syn_dict.items() if len(val)!=0}

Extracted the names of all available synthetic datasets corresponding to 41 real datasets


In [8]:
#import synthetic data into database
for key in tabular_test_set.keys():
    meta_json=key+'.json'
    for syn_table_name in tabular_test_set[key]:
        syn_csv=syn_table_name+'.csv'
        data,_=prep_data_for_db(os.path.join(syn_dir,syn_csv)) # Note: This function  can be used to quickly generate metadata template, but this is not shown here.
        with open(os.path.join(metadata_dir, meta_json), 'r') as f:
            metadata=json.load(f)
        print(f"Processing {syn_csv} and {meta_json} ")
        candidate_db_idx=get_vars_to_index(metadata,data)
        table_name=Path(syn_csv).stem
        make_table(table_name, data, conn,indx_vars=candidate_db_idx)
        

Processing C1_syn_default_19.csv and C1.json 
Table C1_syn_default_19 already exists in the database
Processing C1_syn_default_3.csv and C1.json 
Table C1_syn_default_3 already exists in the database
Processing C1_synthpop_7.csv and C1.json 
Table C1_synthpop_7 already exists in the database
Processing C1_syn_default_16.csv and C1.json 
Table C1_syn_default_16 already exists in the database
Processing C1_syn_default_17.csv and C1.json 
Table C1_syn_default_17 already exists in the database
Processing C1_syn_default_11.csv and C1.json 
Table C1_syn_default_11 already exists in the database
Processing C1_syn_default_15.csv and C1.json 
Table C1_syn_default_15 already exists in the database
Processing C1_synthpop_1.csv and C1.json 
Table C1_synthpop_1 already exists in the database
Processing C1_synthpop_6.csv and C1.json 
Table C1_synthpop_6 already exists in the database
Processing C1_syn_default_9.csv and C1.json 
Table C1_syn_default_9 already exists in the database
Processing C1_syn_

In [9]:
#save tabula test set for late usage in testing fuzzy SQL
with open('tabular_test_set.json','w') as f:
    json.dump(tabular_test_set, f)
